In [53]:
import json
import networkx as nx
import numpy as np

import functions
from importlib import reload
reload(functions)

<module 'functions' from 'c:\\Users\\alsty\\Desktop\\Diploma Thesis\\functions.py'>

In [54]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.MultiDiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"], questionItem=edge["question item"])
    
    return G

In [55]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    interactionGraphIntensity = []
    
    degrees = [0] * n
   
        
    # list of possible peers to inteact for each person
    possiblePeers = functions.analyzeQuiz(prefGraph=preferenceGraph, \
                                          threshold=threshold, \
                                          numNodes=n)
    
    # ------------------------------------- #
    #    interactions at timetsamp t = 0    #
    # ------------------------------------- #
    # 1st step: interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), \
                                                     m=3))

    # 2nd step: add random activities as weights
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    
    # 3rd step: calculate intensity
    snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[0], 
                                                                numNodes=n)
    interactionGraphIntensity.append(snapshotIntensity)

    # 4th step: add snapshot degrees to the total number of degrees
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    # -------------------------------------- #
    #    interactions at timetsamp t >= 1    #
    # -------------------------------------+ #
    for time in range(1, maxTimestamp + 1) :
        
        # 1st step: create new snapshot
        # 2nd step: add activities as weights
        snapshot = functions.createSnapshot(numNodes=n, \
                                            maxInteractions=maxInteractions, \
                                            degrees=degrees, \
                                            possiblePeers=possiblePeers, \
                                            activitiesCount=activitiesCount)
        interactionGraph.append(snapshot)
        
        # 3rd step: calculate intensity
        snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[time], \
                                                                    numNodes=n)
        interactionGraphIntensity.append(snapshotIntensity)
        
        # 4th step: add snapshot degrees to the total number of degrees
        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 
        
    
    # calculate indexes for each snapshot
    graphSequenceIndexes = []
    for time in range(maxTimestamp + 1) :
        snapshotIndexes = functions.calculateIndexes(prefGraph=preferenceGraph, \
                                                     snapshotGraph=interactionGraph[time], \
                                                     numNodes=n)
        graphSequenceIndexes.append(snapshotIndexes)

    
    return {
        "graph" : interactionGraph,
        "sequence intensity": interactionGraphIntensity,
        "individual": {
            "popularity": np.average([item["individual"]["popularity"] for item in graphSequenceIndexes]),
            "antipathy": np.average([item["individual"]["antipathy"] for item in graphSequenceIndexes]),
            "affective connection": np.average([item["individual"]["affective connection"] for item in graphSequenceIndexes]),
            "sociometric status": np.average([item["individual"]["sociometric status"] for item in graphSequenceIndexes]),
            "positive expansion": np.average([item["individual"]["positive expansion"] for item in graphSequenceIndexes]),
            "negative expansion": np.average([item["individual"]["negative expansion"] for item in graphSequenceIndexes]),
            "realistic perception": np.average([item["individual"]["realistic perception"] for item in graphSequenceIndexes])
        },
        "group": {
            "association": np.average([item["group"]["association"] for item in graphSequenceIndexes]),
            "dissociation": np.average([item["group"]["dissociation"] for item in graphSequenceIndexes]),
            "cohesion": np.average([item["group"]["cohesion"] for item in graphSequenceIndexes]),
            "social intensity": np.average([item["group"]["social intensity"] for item in graphSequenceIndexes]),
        }
    }

In [56]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 200,
    "maxTimestamp" : 15
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

print("\033[1mSnapshots added...\033[0m")
for i in range(6) :
    print(f"at t = {i}: {G['graph'][i]}")

print("\n\033[1mGraph Sequence Intensity\033[0m")
print(G['sequence intensity'])

print("\n\033[1mIndividual Indexes\033[0m")
print(G["individual"])

print("\n\033[1mGroup Indexes\033[0m")
print(G["group"])

ValueError: 'a' cannot be empty unless no samples are taken

In [ ]:
timestamp = 1
print(f"\033[1mInteraction activities at timestamp = {timestamp}\033[0m")
for i, j, data in G["graph"][timestamp].edges(data=True) :
    print(f"({i},{j}) => interaction No {data['interaction_index']}: {functions.getActivity(data['interaction_index'])}")

Interaction activities at timestamp = 1
(0,64) => interaction No 4: to be volunteers
(0,11) => interaction No 0: to share content, chat in a social network platform
(0,45) => interaction No 0: to share content, chat in a social network platform
(1,71) => interaction No 4: to be volunteers
(1,77) => interaction No 1: to participate in debate/group discussion & presentation
(1,11) => interaction No 0: to share content, chat in a social network platform
(1,30) => interaction No 4: to be volunteers
(1,56) => interaction No 5: to be teammates in a sports team
(2,35) => interaction No 3: to study in group or work on collaborative projects
(2,85) => interaction No 2: to play games
(2,6) => interaction No 5: to be teammates in a sports team
(3,20) => interaction No 5: to be teammates in a sports team
(3,70) => interaction No 0: to share content, chat in a social network platform
(3,25) => interaction No 5: to be teammates in a sports team
(4,17) => interaction No 4: to be volunteers
(4,64) => 